# Projeção de ocupação de leitos enfermaria e UTI em estados com modelos SEIR e SEAPMDR

Originalmente, a ferramenta **SimulaCovid** utiliza um modelo epidemiológico SEIR (Suscetíveis, Expostos, Infectados e Removidos) para calcular a disseminação e evolução clínica do COVID-19. Esse modelo original era baseado na implementação desenvolvida por [Hill (2020)](https://github.com/alsnhll/SEIR_COVID19) e dividia a população em diferentes "compartimentos"
mutuamente excludentes de indivíduos:

- Suscetíveis ($S$): aqueles que não têm imunidade à doença, estes se tornam
expostos à doença a uma taxa $\\beta_i$ devido ao contato com indivíduos
infetados $I_i$;
- Expostos ($E$): aqueles que entram em contato com a doença e desenvolvem
ou não sintomas, mas ainda não transmitem infecção. Esses indivíduos
expostos progridem para o primeiro estágio de infecção ($I_1$), leve, a
uma taxa $\sigma$.
- Infectados ($I$): são aqueles que desenvolvem sintomas e transmitem a
doença a uma taxa $\beta_i$. Os casos de infecção são separados em
diferentes estágios de gravidade:
- Leve ($I_1$): aquele que não necessita de hospitalização, progride para
    o estado severo a uma taxa $p_1$, ou se recupera a uma taxa $\gamma_1$;
- Severo ($I_2$): aquele que necessita de hospitalização, progride para o
    estado crítico a uma taxa $p_1$, ou se recupera a uma taxa $\gamma_2$;
- Grave ($I_3$): aquele que além de hospitalização, necessita de
    tratamento intensivo com equipamento de ventiladores. Chegando ao caso
    grave, o indivíduo pode se recuperar a uma taxa $\gamma_3$ ou o quadro
    pode levar à morte com uma taxa $\mu$;
- Recuperados ($R$): aqueles que, após o curso da doença, se recuperam e
desenvolvem imunidade - não retornam ao estado suscetível.
- Mortos ($D$): indivíduos que morrem pelo agravamento da infecção.

Conforme a descrição acima, o conjunto de equações que determinavam a dinâmica do modelo é dado por:

$\frac{dS}{dt} = - (\beta_1 I_1 + \beta_2 I_2 + \beta_3 I_3) S$

$\frac{dE}{dt} = (\beta_1 I_1 + \beta_2 I_2 + \beta_3 I_3) S - \sigma E$

$\frac{dI_1}{dt} = \sigma E - (γ_1 + p_1) I_1$

$\frac{dI_2}{dt} = p_1 I_1 - (γ_2 + p_2) I_2$

$\frac{dI_3}{dt} = p_2 I_2 - (γ_3 + μ) I_3$

$\frac{dR}{dt} = γ_1 I_1 + γ_2 I_2 + γ_3 I_3$

$\frac{dD}{dt} = μ I_3$


### Atualizações do modelo

Esse modelo, no entanto, ignora algumas características importantes da transmissão da Covid-19, que foram se tornando mais claras com a evolução do conhecimento sobre a doença. 

Em especial, diversos trabalhos (por exemplo, [Tindale et al., 2020]; [Lee et
  al., 2020]; e [Casey et al., 2020]) têm destacado a existência de transmissão secundária por pacientes pré-sintomáticos (expostos, mas que ainda não desenvolveram sintomas) e assintomáticos (infectados que jamais desenvolverão sintomas). Do ponto de vista da saúde pública, a ausência desses grupos na modelagem epidemiológica pode levar a uma subestimativa a taxa de transmissão no início de um surto local, e a uma superestimativa no número de mortes ([Yang, Lombardi Jr. e Yang, 2020b]).

Para mitigar essa possibilidade, implementamos um modelo mais representativo da história natural da enfermidade. Assim como o modelo original, esse novo modelo foi baseado na implementação proposta por [Hill (2020)] - que, desde março de 2020, também foi alterada para comportar grupos de indivíduos assintomáticos e pré-sintomáticos capazes de transmitir a doença. Com esses novos compartimentos, a implementação se aproxima mais de um modelo SEAPMDR (Suscetíveis \[S], Expostos \[E], Assintomáticos \[A], Pré-sintomáticos \[P], Moderados [M], Severos \[D] e Recuperados \[R]), conforme proposto por [Yang, Lombardi Jr. e Yang (2020a)].

Com os novos compartimentos, o grupo de Expostos ($E$) original passa a ser dividido em um compartimento $E_0$, de indivíduos expostos com o agente infeccioso latente (não transmissível); e um compartimento $E_1$, composto por indivíduos pré-sintomáticos (e transmissíveis):

- Latentes ($E_0$): aqueles que entraram em contato com a doença mas ainda não desenvolveram a doença nem se tornaram transmissíveis. Progridem para o estágio $E_1$ a uma taxa $\sigma_0$.
- Pré-sintomáticos ($E_0$): aqueles que entraram em contato com a doença, ainda não desenvolveram a doença, mas já transmitem o agente infeccioso a uma taxa $\beta_E$. Progridem para o estágio $I_1$ ou $I_0$ *(ver abaixo)* a uma taxa $\sigma_1$.

Além dos casos pré-sintomáticos, foi considerado um novo subcompartimento de infectados: aqueles que, passado o período de incubação, jamais chegam a desenvolver sintomas - mas continuam sendo transmissíveis durante um certo período, antes de se recuperarem completamente. Esse subcompartimento foi denominado $I_0$, e comporta uma proporção $\phi$ dos indivíduos que entram no compartimento $I$ (Infectados):

- Assintomáticos ($I_0$): aqueles indivíduos que já superaram o período de incubação, mas não chegam a demonstrar sintomas. Ainda assim, transmitem a doença a uma taxa $\beta_0$. Se recuperam completamente (e deixam de ser transmissíveis) a uma taxa $\gamma_0$.

Com essas mudanças, as equações atualizadas do modelo passam a ser:


$\frac{dS}{dt} = - (\beta_E E_1 + \beta_1 I_1 + \beta_2 I_2 + \beta_3 I_3) S$

$\frac{dE_0}{dt} = (\beta_E E_1 + \beta_1 I_1 + \beta_2 I_2 + \beta_3 I_3) S - \sigma_0 E_0$

$\frac{dE_1}{dt} = \sigma_0 E_0 - \sigma_1 E_1$

$\frac{dI_0}{dt} = \sigma_1 E_1 \phi - \gamma_0 I_0$

$\frac{dI_1}{dt} = \sigma_1 E_1 (1- \phi) - (\gamma_1 + p_1) I_1$

$\frac{dI_2}{dt} = p_1 I_1 - (\gamma_2 + p_2) I_2$

$\frac{dI_3}{dt} = p_2 I_2 - (\gamma_3 + μ) I_3$

$\frac{dR}{dt} = \gamma_1 I_1 + \gamma_2 I_2 + \gamma_3 I_3$

$\frac{dD}{dt} = \mu I_3$

#### Outras alterações

Outra mudança realizada em relação à implementação original foi a utilização da taxa de transmissão efetiva estimada para o local para calcular a proporção de indivíduos suscetíveis ($S$) são expostos à doença (entram para o compartimento $E_0$) a cada intervalo de tempo. 

Anteriormente, esse cálculo era realizado com uma taxa padrão para a doença - o que ignora as trajetórias potencialmente muito diferentes da epidemia em cada local. Ao invés disso, são utilizadas as estimativas já geradas pelo FarolCovid para as taxas de transmissão locais, seguindo a fórmula (disponibilizada por Wicklin, 2020, entre outros):

$D = \ln(2) / R_t$

onde $D$ é o número de dias para que o número de casos dobre; e $R_t$ é a estimava da taxa de transmissão local.

[Casey et al., 2020]: https://doi.org/10.1101/2020.05.08.20094870
[Hill (2020)]: https://github.com/alsnhll/SEIR_COVID19
[Lee et al., 2020]: https://doi.org/10.1007/s12630-020-01729-x
[Tindale et al., 2020]: https://doi.org/10.7554/eLife.57149
[Yang, Lombardi Jr. e Yang (2020a)]: https://arxiv.org/abs/2004.05715
[Yang, Lombardi Jr. e Yang, 2020b]: https://doi.org/10.1101/2020.10.11.20210831


## Importa bibliotecas

In [15]:
from pathlib import Path

import pandas as pd
import requests
import yaml


# HACK: allows sibling directory imports in notebooks
# CREDIT: https://stackoverflow.com/a/60478241
import sys
sys.path.insert(0, '..') # add parent folder path where lib folder is
from simulacovid import prepare, seir, seapmdr, simulator

## Como rodar a simulação para os estados

### Carrega parâmetros fixos do arquivo de configuração

A ferramenta [**FarolCovid**](https://github.com/ImpulsoGov/farolcovid) disponibiliza um arquivo de configuração que inclui dados dos tempos de progressão da doença, preparados para os cálculos do simulador. 

Como a versão SEAPMDR exige alguns parâmetros inexistentes nas configurações originais, foi disponibilizado [no repositório](https://github.com/bcbernardo/impulsogov-desafio-simulacovid/blob/master/custom_configs.yaml) uma versão modificada com as configurações relevantes.

In [16]:
# Get from GitHub
#url = (
#    "https://raw.githubusercontent.com/bcbernardo/impulsogov-desafio-simulacovid/" +
#    "master/custom_configs.yaml"
#)

#config = yaml.load(requests.get(url).text, Loader=yaml.FullLoader)

# OR get local
with open(Path("..", "custom_configs.yaml"), "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

config["br"]["seir_parameters"]

{'asymptomatic_proportion': 0.3,
 'asymptomatic_duration': 6,
 'mild_duration': 6,
 'severe_duration': 6,
 'critical_duration': 8,
 'fatality_ratio': 0.02,
 'doubling_rate': 1.15,
 'incubation_period': 5,
 'presymptomatic_period': 2,
 'i0_percentage': 0.256,
 'i1_percentage': 0.599,
 'i2_percentage': 0.092,
 'i3_percentage': 0.019,
 'infected_health_care_proportion': 0.05,
 'hospitalized_by_age_perc': {'from_0_to_9': 2e-05,
  'from_10_to_19': 0.0004,
  'from_20_to_29': 0.011,
  'from_30_to_39': 0.034,
  'from_40_to_49': 0.043,
  'from_50_to_59': 0.082,
  'from_60_to_69': 0.118,
  'from_70_to_79': 0.166,
  'from_80_to_older': 0.184}}

### Carrega parâmetros de hospitalização e mortalidade por estado

**Conforme a implementação original**, passamos a utilizar parâmetros calculados por nível geográfico (neste caso estado) levando em consideração a distriuição etária da população. Abaixo segue um trecho de a metodologia com mais detalhes sobre esse cálculo:

> A proporção esperada de hospitalizações é dada pela estimativa de hospitalizações de cada faixa etária, de Verity, Robert, et al. (2020) num amplo estudo com base em 3.665 casos na China, ponderado pelo total da população em cada faixa etária (População residente em 2019 - CNES). Esse valor nos fornece o percentual total de hospitalizações esperadas na população (I2+I3/I). Para obter o percentual por intensidade do caso, mantivemos a razão entre severos (I2) e críticos (I3) constante: antes tínhamos 12.5% severos e 2.5% críticos do total de casos ativos (I) - ou seja, uma razão de 0.2 críticos para cada caso severo (I3/I2) -, que passa a ser de 83.3% severos (I2) e 16.7% críticos (I3) do total estimado de hospitalizados (I2+I3 / I).

Para a incorporação da categoria de assintomáticos, levou-se em conta uma proporção de 30% dos novos casos (assim como faz Hill (2020), citando Bi et al. 2020; Mizumoto et al. 2020; e Nishiura et al. 2020). O 70% restante dos casos corresponde aos indivíduos que desenvolvem a forma moderada da doença, podendo progredir para os próximos estágios. **Ao contrário da implementação original**, aqui calculamos explicitamente o percentual de assintomáticos da população - ao invés de deixar essa tarefa a cargo de uma subrotina dentro do código.

In [17]:
place_id = "state_num_id"

In [18]:
place_specific_params = pd.read_csv("http://datasource.coronacidades.org/br/states/parameters")
place_specific_params.set_index(place_id, inplace=True)

# recalculate severity percentages considering the asymptomatic fraction
asymptomatic_proportion = config["br"]["seir_parameters"]["asymptomatic_proportion"]
place_specific_params["i0_percentage"] = asymptomatic_proportion
place_specific_params[["i1_percentage", "i2_percentage", "i3_percentage"]] = (
    place_specific_params[["i1_percentage", "i2_percentage", "i3_percentage"]]
    .apply(lambda x: x / ( 1 - asymptomatic_proportion))
)

place_specific_params

,fatality_ratio,hospitalized_by_age_perc,i1_percentage,i2_percentage,i3_percentage,data_last_refreshed,i0_percentage
state_num_id,,,,,,,
11,0.004582,0.035272,1.378183,0.041701,0.008688,2021-01-21 00:09:07,0.3
12,0.003882,0.029194,1.386866,0.034515,0.007191,2021-01-21 00:09:07,0.3
13,0.003741,0.029431,1.386528,0.034795,0.007249,2021-01-21 00:09:07,0.3
14,0.003333,0.029069,1.387045,0.034367,0.007160,2021-01-21 00:09:07,0.3
15,0.004362,0.032281,1.382456,0.038165,0.007951,2021-01-21 00:09:07,0.3
16,0.003405,0.028235,1.388235,0.033382,0.006955,2021-01-21 00:09:07,0.3
17,0.005203,0.035553,1.377781,0.042033,0.008757,2021-01-21 00:09:07,0.3
21,0.005125,0.033957,1.380061,0.040146,0.008364,2021-01-21 00:09:07,0.3
22,0.006147,0.039047,1.372790,0.046164,0.009617,2021-01-21 00:09:07,0.3


### Obtém os dados epidemiológicos de entrada

#### Importa dados históricos dos estados

**Ao contrário da implementação original**, utilizamos uma [versão customizada](https://github.com/bcbernardo/coronacidades-datasource) da ferramenta CoronaCidades, com uma [extremidade adicional](https://github.com/bcbernardo/coronacidades-datasource/blob/master/src/loader/endpoints/get_states_farolcovid_history.py) para registrar o histórico de evolução dos dados de:

- Casos e óbitos acumulados até cada data desde os primeiros casos nos estados
- Taxa de notificação e casos ativos estimados, tomando os 15 dias anteriores à data de referência
- Taxa de contágio (Rt), também tomando os 15 dias anteriores

Como a geração desses dados pode levar algumas horas, uma versão consolidada dos dados é disponibilizada localmente no repositório, no caminho `data/br-states-farolcovid-history.csv`, e na cópia do repositório no GitHub.

In [29]:
df_farol = pd.read_csv("../data/br-states-farolcovid-history.csv")

# fix missing ids
state_num_ids = {  # from https://servicodados.ibge.gov.br/api/v1/localidades/estados/
  "RO": 11,
  "AC": 12,
  "AM": 13,
  "RR": 14,
  "PA": 15,
  "AP": 16,
  "TO": 17,
  "MA": 21,
  "PI": 22,
  "CE": 23,
  "RN": 24,
  "PB": 25,
  "PE": 26,
  "AL": 27,
  "SE": 28,
  "BA": 29,
  "MG": 31,
  "ES": 32,
  "RJ": 33,
  "SP": 35,
  "PR": 41,
  "SC": 42,
  "RS": 43,
  "MS": 50,
  "MT": 51,
  "GO": 52,
  "DF": 53,
}
df_farol["state_num_id"] = df_farol["state_id"].apply(lambda UF: state_num_ids[UF])

# erase rows with missing data
df_farol = (
    df_farol
    .dropna()
    .reset_index()
)

df_farol["last_updated_cases"] = pd.to_datetime(df_farol["last_updated_cases"])

df_farol[["state_num_id", "state_id", "last_updated_cases", "confirmed_cases", "deaths", 
          "notification_rate", "active_cases", "rt_most_likely"]]

,state_num_id,state_id,last_updated_cases,confirmed_cases,deaths,notification_rate,active_cases,rt_most_likely
0,35,SP,2020-03-28,1406.0,84.0,0.014600,91849.0,2.84
1,35,SP,2020-03-29,1451.0,98.0,0.013834,100185.0,2.78
2,35,SP,2020-03-30,1517.0,113.0,0.013900,98201.0,2.75
3,35,SP,2020-03-31,2339.0,136.0,0.021356,101843.0,2.61
4,35,SP,2020-04-01,2981.0,164.0,0.027297,100416.0,2.65
...,...,...,...,...,...,...,...,...
7616,43,RS,2021-01-19,512343.0,10051.0,0.515102,106499.0,0.96
7617,50,MS,2021-01-18,152026.0,2705.0,0.343943,45816.0,1.03
7618,51,MT,2021-01-19,202125.0,4846.0,0.211953,46388.0,1.13
7619,52,GO,2021-01-19,332391.0,7107.0,0.251854,38367.0,1.04


### Roda a simulação para cada registro histórico

**Ao contrário da implementação original**, preparamos e rodamos uma simulação para cada modelo (SEIR original e SEAPMDR) e para cada registro epidemiológico diário de cada UF desde o início dos casos. Isso permite avaliar o quanto cada um dos modelos acertou historicamente, comparando seu desempenho.

In [30]:
# create an empty DataFrame to receive predictions
df_predictions = pd.DataFrame()

for _, row in df_farol.iterrows():
    
    # prepare simulation parameters (S, E0, E1, I0, I1, I2, I3, R, D, Rt, N)
    params = prepare.prepare_simulation(row, place_id, config, place_specific_params)
    
    # run one simulation for each model available
    for model in ["SEAPMDR", "SEIR"]:
        
        # run simulation and get (best and worst) scenarios
        dfs_scenarios = simulator.run_simulation(params, config, model=model)
        
        # unpack scenarios
        for scenario in ["worst", "best"]:
            df = dfs_scenarios[scenario]
            df["scenario"] = scenario
            
            df["date_prediction"] = row["last_updated_cases"]
            df["state_num_id"] = row["state_num_id"]
            
            df= (
                df
                .reset_index()
                .rename(columns={"dias": "days"})
            )
            
            # add to DataFrame that collects all predictions
            df_predictions = pd.concat([df_predictions, df], ignore_index=True)

# show results
df_predictions

/mnt/c/Users/bernardo.baron/Projetos/impulsogov-desafio-simulacovid/.venv/lib/python3.8/site-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


,days,S,E0,E1,I0,I1,I2,I3,R,D,N,E,scenario,model,date_prediction,state_num_id
0,1,4.645945e+07,-382022.686501,-254681.791001,27554.700000,125319.820244,4876.996054,1016.040845,4367.000000,84.000000,4.598597e+07,-636704.477502,worst,SEAPMDR,2020-03-28,35
1,2,4.656272e+07,-362716.788717,-250731.435238,-11630.314559,24519.586999,100.522754,946.995765,22589.897472,166.509350,4.598597e+07,-613448.223955,worst,SEAPMDR,2020-03-28,35
2,3,4.671211e+07,-387975.225819,-250040.202112,-44335.782829,-59723.463938,1201.845839,842.550316,13645.365421,240.606290,4.598597e+07,-638015.427931,worst,SEAPMDR,2020-03-28,35
3,4,4.690574e+07,-443727.437406,-260767.904762,-72685.526178,-132585.712030,6492.176962,841.003690,-17649.622955,309.553412,4.598597e+07,-704495.342168,worst,SEAPMDR,2020-03-28,35
4,5,4.714730e+07,-524572.904158,-285516.434544,-99145.679195,-200008.371331,14863.038514,1026.907170,-68363.234751,385.717610,4.598597e+07,-810089.338703,worst,SEAPMDR,2020-03-28,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2774039,87,2.128117e+06,NaN,NaN,NaN,3869.528170,-1713.816479,-497.028746,895728.748764,-2154.829658,3.026064e+06,2714.390975,best,SEIR,2021-01-19,53
2774040,88,2.127652e+06,NaN,NaN,NaN,3768.787879,-1669.434780,-484.305840,896342.348092,-2189.043296,3.026064e+06,2643.416523,best,SEIR,2021-01-19,53
2774041,89,2.127200e+06,NaN,NaN,NaN,3670.611200,-1626.170497,-471.894533,896939.924598,-2222.380639,3.026064e+06,2574.264106,best,SEIR,2021-01-19,53
2774042,90,2.126759e+06,NaN,NaN,NaN,3574.935954,-1583.997127,-459.788030,897521.888542,-2254.863176,3.026064e+06,2506.888712,best,SEIR,2021-01-19,53


In [32]:
df_predictions.to_csv("../data/br-states-simulacovid-predictions.csv", index=False)